In [11]:
import numpy as np
import matplotlib.pyplot as plt
import scienceplots
from tqdm import trange
plt.style.use(['science','grid'])

# import custom modules
import Pub
import PubCrawlFunctions as PCF
import Ant
import Logger
# import randomPubsInit

In [12]:
# simulation paramters
tau0 = 1
alpha = 1
beta = 1
gamma = 1 
rho = 0.2

# simulation counters
time = 0
timeMax = int(60*12)            # 12 hours in minutes - 3pm to 3am 
iter = 0
maxIter = 10000
# population size of ants
popSize = 50

# velocity of an ant
velAnt = int(5000 / 60)         # 5km/h in m/min


In [13]:
Pubs = PCF.initPubs('randomPubs.csv')
# init the pheromone matrix which is a 2D array with the size of the number of pubs
pheromoneMatrix = np.ones((len(Pubs), len(Pubs)))
pheromoneMatrix = pheromoneMatrix * tau0

# init the distance matrix D
distanceMatrix = np.zeros((len(Pubs), len(Pubs)))
for i in range(len(Pubs)):
    for j in range(i, len(Pubs)):
        distanceMatrix[i][j] = PCF.getDistance(Pubs[i], Pubs[j])
        #  print("i and j: ", i, j, " distance: ", distanceMatrix[i][j])

        distanceMatrix[j][i] = distanceMatrix[i][j]

# set the diagonal to 10e15
for i in range(len(Pubs)):
    distanceMatrix[i][i] = 10e15

# init the visibility matrix
visibilityMatrix = 1 / distanceMatrix

In [14]:
# Ant colony

pathCollection = np.zeros((popSize, len(Pubs)))
pathLengthCollection = np.zeros((popSize, 1))
pathDurationCollection = np.zeros((popSize, 1))

minimumPathLength = int(10e15)
minimumPath = np.zeros(len(Pubs))
bestAnt = None


# clear the log file
Logger.clearLog()


## Main algorithm

while(iter < maxIter):
    iter += 1

    # give heartbeat
    if iter % 100 == 0:
        print("Iteration: ", iter)

    # Generate paths
    for i in range(popSize):
        # create an Ant
        ant = Ant.Ant(velocity=1)
        path = PCF.generatePath(pheromoneMatrix, visibilityMatrix, alpha, beta, gamma, Pubs, ant)
        #pathLength = PCF.getPathLength(path, Pubs)
        pathLength = PCF.getPathDuration(ant)
        pathDuration = ant.getTime()
        

        # update the minimal path
        if pathLength < minimumPathLength:
            minimumPathLength = pathLength
            minimumPath = path
            minimumPathTimeTrajectory = ant.timedPath
            bestAnt = ant

            # inform the user
            print('New minimum path: %d found, in iteration: ', minimumPathLength, iter)
            print("Path: ", minimumPath)

            # log the new minimum path
            Logger.logBestPath(minimumPath, minimumPathTimeTrajectory, minimumPathLength)

            # if Plotting:
            #     # update scatter plot
            #     x = [Pubs[pubID].posX for pubID in minimumPath]
            #     y = [Pubs[pubID].posY for pubID in minimumPath]
            #     pubs_scatter.set_offsets(np.column_stack((x, y)))

            #     # update connection lines
            #     connection_lines.set_xdata(x)
            #     connection_lines.set_ydata(y)

            #     # update the pheromone matrix
            #     axs[1].imshow(pheromoneMatrix, cmap='hot', interpolation='nearest')

            #     # update the path length plot
            #     pathLengthPlot.set_xdata(np.append(pathLengthPlot.get_xdata(), iter*(popSize)+(i+1)))
            #     pathLengthPlot.set_ydata(np.append(pathLengthPlot.get_ydata(), minimumPathLength))
            #     axs[2].set_xlim(0, iter*(popSize)+(i+1))
            #     if minimumPathLength > LengthMax:
            #         LengthMax = minimumPathLength
            #         axs[2].set_ylim(0, int(minimumPathLength*1.1))


            #     # update the figure title to the minimal path length and iteration
            #     fig.suptitle('Pub Crawl, iteration: ' + str(iter) + ', path length: ' + str(minimumPathLength))

            #     plt.pause(0.01)





        pathCollection[i,:] = path
        pathLengthCollection[i] = pathLength
        pathDurationCollection[i] = pathDuration

    # update the pheromone matrix
    #deltaPheromoneMatrix = PCF.getDeltaPheromoneMatrix(pathCollection, pathLengthCollection)
    deltaPheromoneMatrix = PCF.getDeltaPheromoneMatrix(pathCollection, pathDurationCollection)
    pheromoneMatrix = PCF.updatePheromoneMatrix(pheromoneMatrix, deltaPheromoneMatrix, rho)




New minimum path: %d found, in iteration:  8004839.761351184 1
Path:  [6, 3, 5, 2, 8, 9, 4, 7, 1, 0]
New minimum path: %d found, in iteration:  7004588.584102965 1
Path:  [5, 3, 8, 9, 6, 2, 0, 1, 4, 7]
New minimum path: %d found, in iteration:  7004359.943597239 1
Path:  [1, 9, 8, 5, 2, 0, 4, 6, 7, 3]
New minimum path: %d found, in iteration:  7003855.688747992 1
Path:  [5, 9, 1, 8, 0, 7, 6, 2, 3, 4]
New minimum path: %d found, in iteration:  6004640.16840956 1
Path:  [8, 5, 9, 1, 4, 6, 0, 3, 2, 7]
New minimum path: %d found, in iteration:  6003870.628156891 1
Path:  [0, 8, 9, 5, 2, 6, 1, 7, 3, 4]
New minimum path: %d found, in iteration:  5004074.688865859 1
Path:  [0, 8, 9, 5, 1, 6, 2, 3, 4, 7]
New minimum path: %d found, in iteration:  5003963.017305098 1
Path:  [0, 9, 5, 8, 1, 6, 2, 3, 4, 7]
New minimum path: %d found, in iteration:  5003766.906084814 2
Path:  [5, 8, 9, 7, 1, 4, 2, 6, 3, 0]
New minimum path: %d found, in iteration:  5003287.899555432 3
Path:  [0, 8, 9, 7, 1, 3, 5, 

In [ ]:
# inform the user over the best found ant
print("Best ant: ", bestAnt)
print("Best path: ", minimumPath)
print("Best path length: ", minimumPathLength)
print("Best path time trajectory: ", minimumPathTimeTrajectory)


Best ant:  <Ant.Ant object at 0x106868850>
Best path:  [0, 8, 9, 5, 1, 3, 4, 6, 2, 7]
Best path length:  5002899.84481841
Best path time trajectory:  [(0, 0), (8, 243.27792503337315), (9, 435.69002932313913), (5, 665.7913029951762), (1, 1042.8486048812706), (3, 1001226.7453150204), (4, 2001748.3294405458), (6, 3002170.327070871), (2, 4002513.2235567085), (7, 5002899.84481841)]
